In [2]:
# Load ckpt model 
import os 
import pickle
from contextlib import nullcontext
import torch
import tiktoken
from model import GPTConfig, GPT

seed = 1337
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device = "cpu"
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32' or 'bfloat16' or 'float16'
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)
device = "cuda"


out_dir = "out-enwiki-char"
device = "cuda"
ckpt_path = os.path.join(out_dir, 'ckpt.pt')
checkpoint = torch.load(ckpt_path, map_location=device)
# gptconf = GPTConfig(**checkpoint['model_args'])
# model = GPT(gptconf)
# state_dict = checkpoint['model']
# unwanted_prefix = '_orig_mod.'
# for k,v in list(state_dict.items()):
#     if k.startswith(unwanted_prefix):
#         state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
# model.load_state_dict(state_dict)
# model.to(device)

/var/folders/nn/nbsb8w3570zfgs23h2s0cdzm0000gn/T/ipykernel_11183/3161367042.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_locat

FileNotFoundError: [Errno 2] No such file or directory: 'out-enwiki-char/ckpt.pt'

In [ ]:
# Load data encoder & decoder
meta_path = os.path.join('data', checkpoint['config']['dataset'], 'meta.pkl')
print(f"Loading meta from {meta_path}...")
with open(meta_path, 'rb') as f:
    meta = pickle.load(f)
# TODO want to make this more general to arbitrary encoder/decoder schemes
stoi, itos = meta['stoi'], meta['itos']
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [ ]:
# Attention Map (Interested in Attention Sink) & Tokenwise Perplexity Plot (Interested in Entropy Spike)
# start = "\n" # or "<|endoftext|>" or etc. Can also specify a file, use as: "FILE:prompt.txt"
start = "Hello!\n"
# encode the beginning of the prompt
if start.startswith('FILE:'):
    with open(start[5:], 'r', encoding='utf-8') as f:
        start = f.read()
start_ids = encode(start)
x = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])

In [ ]:
num_samples = 1
max_new_tokens = 1024
temperature = 1.0
top_k = 10
with torch.no_grad():
    with ctx:
        for k in range(num_samples):
            y = model.generate(x, max_new_tokens, temperature=temperature, top_k=top_k)
            print(decode(y[0].tolist()))
            print('---------------')

In [4]:
txt_file = "data/enwiki/filtered_text.txt"
# load txt file 
with open(txt_file, 'r') as file:
    text = file.read()



In [7]:
text[:200]

' anarchism originated as a term of abuse first used against early working class radicals including the diggers of the english revolution and the sans culottes of the french revolution whilst the term '

MindSpore Dataset

In [3]:
import mindspore.dataset as ds
en_wik9_dataset_dir = "/path/to/en_wik9_dataset"
dataset2 = ds.EnWik9Dataset(dataset_dir=en_wik9_dataset_dir, num_samples=2,
                            shuffle=True)

ValueError: The folder /path/to/en_wik9_dataset does not exist or is not a directory or permission denied!

In [2]:
!pip install mindspore

     |████████████████████████████████| 223.9 MB 449 kB/s eta 0:00:01     |██████████████████▉             | 131.8 MB 344 kB/s eta 0:04:28
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
